<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.1


    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.63


    Uninstalling yfinance-0.2.63:
      Successfully uninstalled yfinance-0.2.63
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-06-25 13:01:49
-------------------
qualified stocks: 93
with latest results: 30
still star stocks: 18
-------------------
Initial Investment:  1.17 C
CY Investment:  1.44 C
Reserve:  6.47 K
Current:  1.34 C
-------------------
Today PnL: 1.26 L (0.95%)
Current PnL: -15.70 L (-10.94%)
CY Booked + Current PnL: -9.21 L (-6.42%)
-------------------
Total profit:  3.22 L
Total loss:  -18.92 L
-------------------
Total Booked + Current PnL: 17.77 L (15.24%)
Total Booked PnL: 33.47 L (28.72%)
Curr Year Booked PnL: 6.49 L (4.86%)
Prev Year Booked PnL: 26.98 L (23.15%)
Est FTT:  2.06 C
Est FTT PnL: 72.59 L (54.33%)
Deployed:  1.17 C
Current:  1.34 C
CAGR/XIRR %: 9.39%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'RSP', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
62,RELIANCE,1291.83,1526.00,7.63,X-LC,50.54,159772.0,18963.0,6567.0,1.03,13.47,4.11,18.13,18.0,2.89,1.20,26.16,XY25,NTT,REFINERIES
84,UJJIVANSFB,52.77,53.00,52.86,M-SC,86.02,132894.0,-9585.0,10206.0,1.01,-6.73,7.68,0.44,242.0,-0.94,1.00,56.85,OX40N,NTT,BANKS
89,VIPIND,488.80,489.00,-975.71,H-SC,97.85,83402.0,-11914.0,11952.0,-1.98,-12.50,14.33,0.04,153.0,-1.00,0.62,64.50,OX40N,NTT,MISC
3,ADANIPORTS,1103.69,1583.00,-1.49,M-LC,38.71,98704.0,20342.0,13690.0,0.05,25.96,13.87,43.43,79.0,1.49,0.74,32.66,XY24,NTT,MISC
35,HINDALCO,651.95,761.55,-10.07,H-LC,49.46,107432.0,3120.0,14417.0,0.31,2.99,13.42,16.81,27.0,0.22,0.80,19.52,X5K,ATH,METALS


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BANKINDIA,116.91,190.00,-33.09,M-MC,53.76,178792.0,-1016.0,113426.0,0.85,-0.56,63.44,62.52,170.0,-0.01,1.34,33.33,XR,NTT,BANKS
50,KPIGREEN,497.21,732.16,NaN,NaN,75.27,125660.0,363.0,58847.0,1.08,0.29,46.83,47.25,137.0,0.01,0.94,57.21,MH,ATH,POWER
34,HEROMOTOCO,4311.81,6039.03,-1.43,H-MC,34.41,150570.0,-343.0,60800.0,0.22,-0.23,40.38,40.06,102.0,-0.01,1.13,23.16,AR,ATH,AUTO
56,NATIONALUM,189.63,247.44,-36.43,H-SC,55.91,100302.0,-202.0,30843.0,-0.26,-0.20,30.75,30.49,97.0,-0.01,0.75,32.14,MH,ATH,METALS


In [11]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
66,SAMMAANCAP,170.35,326.00,-172.08,M-SC,95.70,87840.0,-14370.0,107762.0,17.42,-14.06,122.68,91.37,203.0,-0.13,0.66,42.59,XY25,NTT,FINANCE
25,DEN,52.79,75.00,122.80,L-SC,87.10,62512.0,-21952.0,57486.0,10.34,-25.99,91.96,42.07,257.0,-0.38,0.47,32.76,AR,NTT,ENTERTAINMENT
31,HATHWAY,19.74,31.40,1030.43,L-SC,74.19,64880.0,-14080.0,60721.0,6.99,-17.83,93.59,59.07,262.0,-0.23,0.49,30.18,XR,NTT,ENTERTAINMENT
4,ALKYLAMINE,2347.98,4546.37,7.24,X-SC,100.00,99889.0,-1074.0,95604.0,5.70,-1.06,95.71,93.63,65.0,-0.01,0.75,48.78,SR,ATH,CHEMICALS
41,INDIAMART,2327.09,4850.92,-46.43,H-SC,92.47,139655.0,16319.0,117450.0,5.62,13.23,84.10,108.45,121.0,0.14,1.05,38.54,AR,ATH,MISC


In [12]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
89,VIPIND,488.80,489.00,-975.71,H-SC,97.85,83402.0,-11914.0,11952.0,-1.98,-12.50,14.33,0.04,153.0,-1.00,0.62,64.50,OX40N,NTT,MISC
85,UNIONBANK,123.87,163.00,-6.44,M-LC,61.29,165115.0,24275.0,20210.0,-1.27,17.24,12.24,31.59,90.0,1.20,1.24,43.43,XY24,NTT,BANKS
71,SIEMENS,4406.90,7969.85,-13.85,X-LC,21.51,78000.0,-32172.0,121243.0,-1.26,-29.20,155.44,80.85,12.0,-0.27,0.58,14.88,AR,ATH,ELECTRICAL
87,VALIANTORG,512.64,838.00,-357.64,H-SC,76.34,109460.0,-23826.0,108420.0,-1.25,-17.88,99.05,63.47,145.0,-0.22,0.82,83.48,XR,NTT,CHEMICALS
65,SAIL,130.64,228.00,95.22,M-MC,46.24,222396.0,-2566.0,170222.0,-1.22,-1.14,76.54,74.53,186.0,-0.02,1.67,29.12,XY24,BTT,STEEL


In [13]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,UJJIVANSFB,52.77,53.0,52.86,M-SC,86.02,132894.0,-9585.0,10206.0,1.01,-6.73,7.68,0.44,242.0,-0.94,1.00,56.85,OX40N,NTT,BANKS
89,VIPIND,488.80,489.0,-975.71,H-SC,97.85,83402.0,-11914.0,11952.0,-1.98,-12.50,14.33,0.04,153.0,-1.00,0.62,64.50,OX40N,NTT,MISC
78,TATAELXSI,7332.28,7332.0,-16.48,X-MC,33.33,87752.0,-14900.0,14900.0,0.03,-14.51,16.98,-0.00,58.0,-1.00,0.66,33.39,OX40N,NTT,IT
42,INDIGOPNTS,1407.73,1408.0,122.10,H-SC,89.25,145700.0,-28859.0,28892.0,1.05,-16.53,19.83,0.02,128.0,-1.00,1.09,26.34,OX40N,NTT,PAINTS
76,SYMPHONY,1306.42,1306.0,-33.55,M-SC,2.15,141650.0,-29491.0,29435.0,0.68,-17.23,20.78,-0.03,188.0,-1.00,1.06,1.81,OX40N,NTT,DURABLES


In [14]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
80,TCS,3794.03,4998.00,-18.41,X-LC,26.88,266276.0,-29658.0,123579.0,0.69,-10.02,46.41,31.73,1.0,-0.24,1.99,6.58,X40,BTT,IT
44,INFY,1461.46,2275.00,-9.91,X-LC,60.22,288441.0,26840.0,118780.0,1.88,10.26,41.18,55.67,2.0,0.23,2.16,16.99,X40,BTT,IT
36,HINDUNILVR,2413.81,2723.00,-18.89,X-LC,20.43,254778.0,-15569.0,50191.0,0.52,-5.76,19.70,12.81,6.0,-0.31,1.91,6.55,XY25,NTT,FMCG
15,BRITANNIA,4983.33,6446.05,14.50,X-LC,59.14,262637.0,33404.0,33880.0,1.65,14.57,12.90,29.35,7.0,0.99,1.97,24.79,XY25,ATH,FMCG
57,NESTLEIND,2268.60,2755.00,-11.58,X-LC,30.11,280695.0,15269.0,41627.0,1.41,5.75,14.83,21.44,9.0,0.37,2.10,12.31,XY25,NTT,FMCG


In [15]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
3,ADANIPORTS,1103.69,1583.0,-1.49,M-LC,38.71,98704.0,20342.0,13690.0,0.05,25.96,13.87,43.43,79.0,1.49,0.74,32.66,XY24,NTT,MISC


In [16]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,CAMS,3643.00,5250.99,5.78,H-SC,78.49,118994.0,16990.0,28035.0,0.54,16.66,23.56,44.14,124.0,0.61,0.89,35.60,X40N,ATH,MISC
3,ADANIPORTS,1103.69,1583.00,-1.49,M-LC,38.71,98704.0,20342.0,13690.0,0.05,25.96,13.87,43.43,79.0,1.49,0.74,32.66,XY24,NTT,MISC
92,WIPRO,243.46,420.00,-2.90,M-LC,79.57,166656.0,15711.0,93744.0,1.45,10.41,56.25,72.51,69.0,0.17,1.25,14.80,XR,NTT,IT
85,UNIONBANK,123.87,163.00,-6.44,M-LC,61.29,165115.0,24275.0,20210.0,-1.27,17.24,12.24,31.59,90.0,1.20,1.24,43.43,XY24,NTT,BANKS
39,ICICIGI,1839.64,2260.25,-9.73,H-MC,83.87,148688.0,12555.0,18571.0,1.13,9.22,12.49,22.86,35.0,0.68,1.11,23.00,X40,ATH,INSURANCE


In [17]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
91,WHIRLPOOL,1167.49,2270.00,-30.12,M-SC,82.80,203595.0,31974.0,130097.0,0.76,18.63,63.90,94.43,196.0,0.25,1.52,50.27,XR,NTT,DURABLES
92,WIPRO,243.46,420.00,-2.90,M-LC,79.57,166656.0,15711.0,93744.0,1.45,10.41,56.25,72.51,69.0,0.17,1.25,14.80,XR,NTT,IT
41,INDIAMART,2327.09,4850.92,-46.43,H-SC,92.47,139655.0,16319.0,117450.0,5.62,13.23,84.10,108.45,121.0,0.14,1.05,38.54,AR,ATH,MISC
50,KPIGREEN,497.21,732.16,NaN,NaN,75.27,125660.0,363.0,58847.0,1.08,0.29,46.83,47.25,137.0,0.01,0.94,57.21,MH,ATH,POWER
12,BANKINDIA,116.91,190.00,-33.09,M-MC,53.76,178792.0,-1016.0,113426.0,0.85,-0.56,63.44,62.52,170.0,-0.01,1.34,33.33,XR,NTT,BANKS


In [18]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,COFFEEDAY,59.14,80.00,-49.70,L-SC,96.77,73056.0,-40493.0,80544.0,1.93,-35.66,110.25,35.27,261.0,-0.50,0.55,78.81,XR,NTT,HOTELS
66,SAMMAANCAP,170.35,326.00,-172.08,M-SC,95.70,87840.0,-14370.0,107762.0,17.42,-14.06,122.68,91.37,203.0,-0.13,0.66,42.59,XY25,NTT,FINANCE
54,MASFIN,326.60,402.39,-19.15,H-SC,70.97,91785.0,-6195.0,28931.0,1.78,-6.32,31.52,23.21,141.0,-0.21,0.69,33.12,XR,ATH,FINANCE
3,ADANIPORTS,1103.69,1583.00,-1.49,M-LC,38.71,98704.0,20342.0,13690.0,0.05,25.96,13.87,43.43,79.0,1.49,0.74,32.66,XY24,NTT,MISC
4,ALKYLAMINE,2347.98,4546.37,7.24,X-SC,100.00,99889.0,-1074.0,95604.0,5.70,-1.06,95.71,93.63,65.0,-0.01,0.75,48.78,SR,ATH,CHEMICALS


In [19]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
4,ALKYLAMINE,2347.98,4546.37,7.24,X-SC,100.00,99889.0,-1074.0,95604.0,5.70,-1.06,95.71,93.63,65.0,-0.01,0.75,48.78,SR,ATH,CHEMICALS
7,ASIANTILES,75.41,137.00,7211.11,M-SC,98.92,80238.0,-13572.0,90188.0,-0.80,-14.47,112.40,81.67,240.0,-0.15,0.60,55.38,XR,NTT,CERAMICS
89,VIPIND,488.80,489.00,-975.71,H-SC,97.85,83402.0,-11914.0,11952.0,-1.98,-12.50,14.33,0.04,153.0,-1.00,0.62,64.50,OX40N,NTT,MISC
22,COFFEEDAY,59.14,80.00,-49.70,L-SC,96.77,73056.0,-40493.0,80544.0,1.93,-35.66,110.25,35.27,261.0,-0.50,0.55,78.81,XR,NTT,HOTELS
66,SAMMAANCAP,170.35,326.00,-172.08,M-SC,95.70,87840.0,-14370.0,107762.0,17.42,-14.06,122.68,91.37,203.0,-0.13,0.66,42.59,XY25,NTT,FINANCE


In [20]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.13
1,25,42.36
2,50,71.09


In [21]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    38.87
LC    37.04
MC    21.50
Name: CurrAlloc%, dtype: float64

In [22]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     31.13
XY25     13.40
X40      12.40
X40N     10.63
XR       10.39
OX40N     8.34
AR        7.90
MH        1.69
X200      1.47
X5K       1.46
SR        1.24
Name: CurrAlloc%, dtype: float64

In [23]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    19.51
X-LC    15.33
M-SC    14.04
H-LC    12.37
M-LC     8.25
H-MC     8.10
X-MC     7.49
M-MC     5.55
X-SC     2.80
L-SC     2.52
L-LC     1.09
L-MC     0.36
Name: CurrAlloc%, dtype: float64

In [24]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.34,-2.25,36.46
IT,12.03,-10.62,65.55
FINANCE,7.96,-13.57,54.68
BANKS,7.95,-7.61,51.52
MISC,6.73,-20.39,67.34
PAINTS,5.62,-16.21,39.11
ELECTRICAL,5.04,-4.33,49.51
HEALTHCARE,4.50,-7.34,38.70
AUTO,4.40,-15.09,65.70


In [25]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2774409.0,25
XR,1073930.0,13
AR,1015328.0,9
X40,663089.0,9
X40N,511449.0,10
XY25,497650.0,9
OX40N,317747.0,11
SR,179164.0,2
MH,89690.0,2


In [26]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,1891892.0,19
M-SC,1455360.0,18
X-LC,617213.0,9
M-MC,603633.0,5
H-LC,601535.0,11
H-MC,438289.0,6
X-MC,417180.0,6
M-LC,343227.0,6
L-SC,261987.0,4


In [27]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,,sum,count
Conviction,Criteria,,
H-SC,XY24,870591.0,6
M-SC,XY24,595754.0,5
H-SC,AR,516486.0,3
M-MC,XY24,398092.0,3
M-SC,XR,333835.0,4
X-LC,X40,317972.0,3
H-LC,X40N,238192.0,5
M-LC,XY24,233290.0,4
H-MC,XY24,231758.0,2


In [28]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 29.0 seconds
